# TSA Chapter 3: ADF Unit Root Test

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_adf_test/TSA_ch3_adf_test.ipynb)

This notebook demonstrates:
- Augmented Dickey-Fuller test on four series: stationary AR(1), random walk, trend+noise, differenced RW
- H0: unit root.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 300

# Generate test series
eps = np.random.normal(0, 1, n)
stationary = np.zeros(n)
for t in range(1, n):
    stationary[t] = 0.5 * stationary[t-1] + eps[t]

random_walk = np.cumsum(eps)
trend_stationary = 0.05 * np.arange(n) + np.random.normal(0, 1, n)

series_list = [stationary, random_walk, trend_stationary, np.diff(random_walk)]
titles = ['Stationary AR(1)', 'Random Walk', 'Trend + Noise', 'Differenced RW']
colors = ['#2E7D32', '#DC3545', '#E67E22', '#1A3A6E']

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, s, title, c in zip(axes.flatten(), series_list, titles, colors):
    ax.plot(s, color=c, linewidth=0.8)
    ax.axhline(y=np.mean(s), color='gray', linestyle='--', linewidth=0.5)

    adf_result = adfuller(s, regression='c')
    stat, pval = adf_result[0], adf_result[1]
    decision = 'Stationary' if pval < 0.05 else 'Unit Root'
    box_color = 'lightgreen' if pval < 0.05 else 'lightyellow'

    ax.set_title(title, fontweight='bold')
    ax.text(0.02, 0.98,
            f'ADF stat: {stat:.3f}\np-value: {pval:.4f}\n{decision}',
            transform=ax.transAxes, fontsize=9, va='top',
            bbox=dict(boxstyle='round', facecolor=box_color, alpha=0.8))
    ax.set_xlabel('Time')

plt.suptitle('ADF Unit Root Test: $H_0$: Unit Root vs $H_1$: Stationary',
             fontweight='bold', fontsize=12)
plt.tight_layout()
save_chart(fig, 'ch3_adf_test')
plt.show()

print("ADF Test Summary:")
print("  H0: Series has a unit root (non-stationary)")
print("  H1: Series is stationary")
print("  Reject H0 if p-value < 0.05")